In [2]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/fedesoriano/cifar100")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Dataset URL: https://www.kaggle.com/datasets/fedesoriano/cifar100


100%|██████████| 161M/161M [00:13<00:00, 12.3MB/s] 


In [35]:
import pickle
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import torchvision.models as models
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)


cuda


In [65]:
train_file =r"/home/namankarki/Naman/3_months_of_data-science/Deep_Learning_Pytorch/cnn/cifar100/train"
test_data = r"/home/namankarki/Naman/3_months_of_data-science/Deep_Learning_Pytorch/cnn/cifar100/test"

def load_data(filename):
    with open (filename, "rb") as f:
        data = pickle.load(f, encoding="bytes")
        images = data[b"data"]
        labels = data[b"fine_labels"]
        images = images.reshape(-1,3,32,32)
        labels = np.array(labels)
        return images, labels

In [66]:
train_images, train_labels = load_data(train_file)
test_images, test_labels = load_data(test_data)


In [75]:
print(train_images.shape)
print(test_images.shape)

(50000, 3, 32, 32)
(10000, 3, 32, 32)


In [70]:
train_transform = transforms.Compose([
    transforms.ToPILImage(),  # converts (3,32,32) to PIL Image
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),    # converts to float32 tensor [0,1]
    transforms.Normalize(mean=[0.5071,0.4867,0.4408], std=[0.2675,0.2565,0.2761])
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5071,0.4867,0.4408], std=[0.2675,0.2565,0.2761])
])


In [92]:
class CustomDataset(Dataset):
    def __init__(self, images, labels, transformation = None):
        self.images = images
        self.labels = labels
        self.transformation = transformation
        
    def __len__(self):
        return(len(self.images))
    
    def __getitem__(self, idx):
        
        image = self.images[idx]
        label = self.labels[idx]
        image = np.transpose(image, (1,2,0))
        if self.transformation:
            image = self.transformation(image)
            
        label = torch.tensor(label, dtype = torch.long)
        
        return image , label

In [93]:
training_dataset = CustomDataset(train_images,train_labels,train_transform)
testing_dataset = CustomDataset(test_images, test_labels,test_transform)

In [94]:
training_dataloader = DataLoader(training_dataset , batch_size = 64, shuffle = True)
testing_dataloader = DataLoader(testing_dataset , batch_size = 64, shuffle = False)

In [95]:
next(iter(training_dataloader))

[tensor([[[[-0.0339, -0.0339, -0.0925,  ..., -0.1512, -0.2391, -0.6789],
           [-0.0925, -0.0192, -0.0485,  ..., -0.3711, -0.1512, -0.0192],
           [-0.3711, -0.0192,  0.0248,  ..., -0.5763, -0.5470, -0.2538],
           ...,
           [-1.3826, -1.4266, -1.4412,  ...,  0.2886,  0.1420,  0.0394],
           [-1.3679, -1.4119, -1.4266,  ...,  0.2447,  0.0981,  0.0394],
           [-1.3679, -1.4119, -1.4412,  ...,  0.2007,  0.0541, -0.0046]],
 
          [[ 0.0289, -0.1240, -0.2310,  ..., -0.3992, -0.5215, -0.8273],
           [ 0.1971,  0.2277,  0.0442,  ..., -0.2769, -0.3533, -0.3074],
           [-0.0781,  0.3347,  0.3653,  ..., -0.1393, -0.2769, -0.2616],
           ...,
           [-1.3776, -1.4235, -1.4235,  ...,  0.9004,  0.7169,  0.6099],
           [-1.3624, -1.4082, -1.4235,  ...,  0.8392,  0.6863,  0.6405],
           [-1.3624, -1.3929, -1.4235,  ...,  0.7628,  0.6405,  0.5793]],
 
          [[-0.5171, -0.8153, -1.0852,  ..., -0.6591, -0.7017, -0.8153],
           [ 

In [96]:
data = iter(training_dataloader)
img, lbl = next(data)
print(img.shape)
print(lbl.shape)

torch.Size([64, 3, 32, 32])
torch.Size([64])


In [97]:
resnet = models.resnet50(pretrained=True)

In [98]:
num_ftrs = resnet.fc.in_features
resnet_fc = nn.Linear(num_ftrs, 100)

In [99]:
resnet = resnet.to(device)

In [100]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(params = resnet.parameters(), lr = 0.001)

In [101]:
train_losses, train_accs = [], []
val_losses, val_accs = [], []

In [102]:
epochs = 10

train_losses, train_accs = [], []
val_losses, val_accs = [], []

for epoch in range(epochs):
    train_loss = 0
    train_acc = 0
    
    resnet.train()
    for image, label in training_dataloader:
        image = image.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        logits = resnet(image)
        batch_loss = criterion(logits, label)
        batch_loss.backward()
        optimizer.step()

        train_loss += batch_loss.item() * label.size(0)
        preds = logits.argmax(dim=1)
        train_acc += (preds == label).sum().item()

    avg_loss = train_loss / len(training_dataset)
    avg_acc  = train_acc / len(training_dataset)

    train_losses.append(avg_loss)
    train_accs.append(avg_acc)

    # Validation
    val_loss = 0
    val_acc = 0

    resnet.eval()
    with torch.no_grad():
        for image, label in testing_dataloader:
            image = image.to(device)
            label = label.to(device)

            logits = resnet(image)
            batch_loss = criterion(logits, label)

            val_loss += batch_loss.item() * label.size(0)
            preds = logits.argmax(dim=1)
            val_acc += (preds == label).sum().item()

    avg_val_loss = val_loss / len(testing_dataset)
    avg_val_acc  = val_acc / len(testing_dataset)

    val_losses.append(avg_val_loss)
    val_accs.append(avg_val_acc)

    print(
        f"Epoch [{epoch+1}/{epochs}] "
        f"Train Loss: {avg_loss:.4f}, Train Acc: {avg_acc:.4f} | "
        f"Val Loss: {avg_val_loss:.4f}, Val Acc: {avg_val_acc:.4f}"
    )


Epoch [1/10] Train Loss: 3.8681, Train Acc: 0.1509 | Val Loss: 3.1151, Val Acc: 0.2808
Epoch [2/10] Train Loss: 2.9225, Train Acc: 0.2835 | Val Loss: 2.8688, Val Acc: 0.3448
Epoch [3/10] Train Loss: 2.6346, Train Acc: 0.3355 | Val Loss: 2.2162, Val Acc: 0.4104
Epoch [4/10] Train Loss: 2.3989, Train Acc: 0.3764 | Val Loss: 2.1591, Val Acc: 0.4246
Epoch [5/10] Train Loss: 2.6172, Train Acc: 0.3378 | Val Loss: 2.2141, Val Acc: 0.4213
Epoch [6/10] Train Loss: 2.1380, Train Acc: 0.4247 | Val Loss: 2.0518, Val Acc: 0.4563
Epoch [7/10] Train Loss: 1.9848, Train Acc: 0.4597 | Val Loss: 1.9449, Val Acc: 0.4852
Epoch [8/10] Train Loss: 1.8425, Train Acc: 0.4907 | Val Loss: 2.1876, Val Acc: 0.4356
Epoch [9/10] Train Loss: 1.8566, Train Acc: 0.4910 | Val Loss: 1.9309, Val Acc: 0.4981
Epoch [10/10] Train Loss: 1.7371, Train Acc: 0.5153 | Val Loss: 1.7932, Val Acc: 0.5121


Freezing layers can be done to imporve the result and learning rate decay can be used